In [21]:
from tsplearn import *
import numpy as np 
import pandas as pd

prob_T=0.8

# Load the graph
G = EnhancedGraph(n=40, p_edges=0.162, p_triangles=prob_T, seed=0)
B1 = G.get_b1()
B2 = G.get_b2()

# Sub-sampling if needed to decrease complexity
sub_size = 100
B1 = B1[:, :sub_size]
B2 = B2[:sub_size, :]
B2 = B2[:,np.sum(np.abs(B2), 0) == 3]
nu = B2.shape[1]
nd = B1.shape[1]
T = int(np.ceil(nu*(1-prob_T)))

# Laplacians
Lu, Ld, L = G.get_laplacians(sub_size=100)
M =  L.shape[0]


# Problem and Dictionary Dimensionalities
dictionary_type="separated"
m_train = 150 # Number of Train Signals
m_test = 80 # Number of Test Signal
P = 3 # Number of Kernels (Sub-dictionaries)
J = 2 # Polynomial order
sparsity = .1 # Sparsity percentage
K0_max = 20 #floor(M*sparsity) # Sparsity
sparsity_mode = "max"
n_search = 1500
n_sim = 10

# Data-Independent Problem Hyperparameters
K0_coll = np.arange(5, 26, 4) 
max_iter = 100 
patience = 5 
tol = 1e-s # tolerance for Patience
lambda_ = 1e-s # l2 multiplier
verbose = True

In [2]:
D_true, Y_train, Y_test, X_train, X_test, epsilon_true, c_true = generate_data(dictionary_type=dictionary_type,
                                                                                Lu=Lu,
                                                                                Ld=Ld,
                                                                                M=M,
                                                                                P=P,
                                                                                J=J,
                                                                                n_sim=n_sim,
                                                                                m_test=m_test,
                                                                                m_train=m_train,
                                                                                K0_max=K0_max,
                                                                                n_search=n_search,
                                                                                sparsity_mode=sparsity_mode,
                                                                                )

100%|██████████| 1500/1500 [02:14<00:00, 11.15it/s]


...Done! # Best Sparsity: 5


100%|██████████| 1500/1500 [02:14<00:00, 11.17it/s]


...Done! # Best Sparsity: 6


100%|██████████| 1500/1500 [02:14<00:00, 11.11it/s]


...

...Done! # Best Sparsity: 5


100%|██████████| 1500/1500 [01:13<00:00, 20.36it/s]


...Done! # Best Sparsity: 6


100%|██████████| 1500/1500 [01:13<00:00, 20.40it/s]

...Done! # Best Sparsity: 6


In [36]:
f'top_data_T{int(prob_T*100)}'

'top_data_T80'

In [38]:
import os
import pickle 

name = f'top_data_T{int(prob_T*100)}'
save_var = {"D_true" : D_true,
            "Y_train" : Y_train,
            "Y_test" : Y_test,
            "X_train" : X_train,
            "X_test" : X_test,
            "epsilon_true" : epsilon_true,
            "c_true" : c_true}

PATH = os.getcwd()
DIR_PATH = f'{PATH}\\synthetic_data'
FILENAME = f'{DIR_PATH}\\{name}.pkl'

if not os.path.exists(DIR_PATH):
    os.makedirs(DIR_PATH)
    
with open(FILENAME, 'wb') as f: 
    pickle.dump(save_var, f)
f.close()

In [40]:
import dill
import os

path = os.getcwd()
dill.load_session(path+'\\results\\joint\\ipynb_env.db')

In [26]:
import scipy.linalg as sla
import numpy as np
import numpy.linalg as la
import cvxpy as cp
from tsplearn.data_gen import *
from typing import Tuple

def topological_dictionary_learn(Y_train: np.ndarray,
                                 Y_test: np.ndarray, 
                                 J: int, 
                                 M: int, 
                                 P: int,
                                 D0: np.ndarray, 
                                 X0: np.ndarray, 
                                 Lu: np.ndarray, 
                                 Ld: np.ndarray,
                                 dictionary_type: str, 
                                 c: float, 
                                 epsilon: float, 
                                 K0: int,
                                 lambda_: float = 1e-3, 
                                 max_iter: int = 10, 
                                 patience: int = 10,
                                 tol: float = 1e-7, 
                                 verbose: bool = False) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Dictionary learning algorithm implementation for sparse representations of a signal on complex regular cellular.
    The algorithm consists of an iterative alternating optimization procedure defined in two steps: the positive semi-definite programming step
    for obtaining the coefficients and dictionary based on Hodge theory, and the Orthogonal Matching Pursuit step for constructing 
    the K0-sparse solution from the dictionary found in the previous step, which best approximates the original signal.
    Args:
        Y_train (np.ndarray): Training data.
        Y_test (np.ndarray): Testing data.
        J (int): Max order of the polynomial for the single sub-dictionary.
        M (int): Number of data points (number of nodes in the data graph).
        P (int): Number of kernels (sub-dictionaries).
        D0 (np.ndarray): Initial dictionary.
        X0 (np.ndarray): Initial sparse representation.
        Lu (np.ndarray): Upper Laplacian matrix
        Ld (np.ndarray): Lower Laplacian matrix
        dictionary_type (str): Type of dictionary.
        c (float): Boundary constant from the synthetic data generation process.
        epsilon (float): Boundary constant from the synthetic data generation process.
        K0 (int): Sparsity of the signal representation.
        lambda_ (float, optional): Regularization parameter. Defaults to 1e-3.
        max_iter (int, optional): Maximum number of iterations. Defaults to 10.
        patience (int, optional): Patience for early stopping. Defaults to 10.
        tol (float, optional): Tolerance value. Defaults to 1e-s.
        verbose (int, optional): Verbosity level. Defaults to 0.

    Returns:
        Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
         minimum training error, minimum testing error, optimal coefficients, optimal testing sparse representation, and optimal training sparse representation.
    """

    # Define hyperparameters
    min_error_train_norm, min_error_test_norm = 1e20, 1e20
    m_test, m_train = Y_test.shape[1], Y_train.shape[1]
    iter_, pat_iter = 1, 0

    if dictionary_type != "fourier":
        if dictionary_type=="joint":
            Lk, _, _ = compute_Lk_and_lambdak(Lu + Ld, J)
        elif dictionary_type=="edge_laplacian":
            Lk, _, _ = compute_Lk_and_lambdak(Ld, J)
        elif dictionary_type=="separated":
            Luk, _, _ = compute_Lk_and_lambdak(Lu, J, separated=True)
            Ldk, _, _ = compute_Lk_and_lambdak(Ld, J, separated=True)

        # Init the dictionary and the sparse representation 
        D_coll = [cp.Constant(D0[:,(M*i):(M*(i+1))]) for i in range(P)]
        Y = cp.Constant(Y_train)
        X_train = X0
        
        while pat_iter < patience and iter_ <= max_iter:
            
            # SDP Step
            # Init constants and parameters
            D_coll = [cp.Constant(np.zeros((M, M))) for i in range(P)] 
            Dsum = cp.Constant(np.zeros((M, M)))
            X = cp.Constant(X_train)
            I = cp.Constant(np.eye(M))
            
            # Define the objective function
            if dictionary_type in ["joint", "edge_laplacian"]:
                # Init the variables
                h = cp.Variable((P, J))
                hI = cp.Variable((P, 1))
                for i in range(0,P):
                    tmp =  cp.Constant(np.zeros((M, M)))
                    for j in range(0,J):
                        tmp += (cp.Constant(Lk[j, :, :]) * h[i,j])
                    tmp += (I*hI[i])
                    D_coll[i] = tmp
                    Dsum += tmp
                D = cp.hstack([D_coll[i]for i in range(P)])
                term1 = cp.square(cp.norm((Y - D @ X), 'fro'))
                term2 = cp.square(cp.norm(h, 'fro')*lambda_)
                term3 = cp.square(cp.norm(hI, 'fro')*lambda_)
                obj = cp.Minimize(term1 + term2 + term3)

            else:
                # Init the variables
                hI = cp.Variable((P, J))
                hS = cp.Variable((P, J))
                hH = cp.Variable((P, 1))
                for i in range(0,P):
                    tmp =  cp.Constant(np.zeros((M, M)))
                    for j in range(0,J):
                        tmp += ((cp.Constant(Luk[j, :, :])*hS[i,j]) + (cp.Constant(Ldk[j, :, :])*hI[i,j]))
                    tmp += (I*hH[i])
                    D_coll[i] = tmp
                    Dsum += tmp
                D = cp.hstack([D_coll[i]for i in range(P)])
                
                term1 = cp.square(cp.norm((Y - D @ X), 'fro'))
                term2 = cp.square(cp.norm(hI, 'fro')*lambda_)
                term3 = cp.square(cp.norm(hS, 'fro')*lambda_)
                term4 = cp.square(cp.norm(hH, 'fro')*lambda_)
                obj = cp.Minimize(term1 + term2 + term3 + term4)

            # Define the constraints
            constraints = [D_coll[i] >> 0 for i in range(P)] + \
                            [(cp.multiply(c, I) - D_coll[i]) >> 0 for i in range(P)] + \
                            [(Dsum - cp.multiply((c - epsilon), I)) >> 0, (cp.multiply((c + epsilon), I) - Dsum) >> 0]

            prob = cp.Problem(obj, constraints)
            prob.solve(solver=cp.MOSEK, verbose=False)
            # Update the dictionary
            D = D.value

            # OMP Step
            dd = la.norm(D, axis=0)
            W = np.diag(1. / dd)
            Domp = D @ W
            X_train = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_train)
            X_test = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_test)
            # Normalization
            X_train = W @ X_train
            X_test = W @ X_test

            # Error Updating
            error_train_norm = (1/m_train)* np.sum(la.norm(Y_train - (D @ X_train), axis=0)**2 /
                                    la.norm(Y_train, axis=0)**2)
            error_test_norm = (1/m_test)* np.sum(la.norm(Y_test - (D @ X_test), axis=0)**2 /
                                    la.norm(Y_test, axis=0)**2)

            
            # Error Storing
            if (error_train_norm < min_error_train_norm) and (abs(error_train_norm) > np.finfo(float).eps) and (abs(error_train_norm - min_error_train_norm) > tol):
                X_opt_train = X_train
                min_error_train_norm = error_train_norm

            if (error_test_norm < min_error_test_norm) and (abs(error_test_norm) > np.finfo(float).eps) and (abs(error_test_norm - min_error_test_norm) > tol):
                h_opt = h.value if dictionary_type in ["joint", "edge_laplacian"] else [hS.value, hI.value, hH.value]
                D_opt = D
                X_opt_test = X_test
                min_error_test_norm = error_test_norm
                pat_iter = 0
                if verbose == 1:
                    print("New Best Test Error:", min_error_test_norm)
            else:
                pat_iter += 1

            # print("-"*100)
            # print(f'Iter: {iter_}')
            # print()
            # print(f'hH.shape: {hH.shape}')
            # print(f'hH: {hH.value}')
            # print()
            # print(f'hS.shape: {hS.shape}')
            # print(f'hS: {hS.value}')
            # print()
            # print(f'hI.shape: {hI.shape}')
            # print(f'hI: {hI.value}')
            # print()
            # print(f'test error: {error_test_norm}')
            # print()
            # print("-"*100)

            iter_ += 1
    
    else:
        # Fourier Dictionary Benchmark
        L = Lu + Ld
        _, D_opt = sla.eigh(L)
        dd = la.norm(D_opt, axis=0)
        W = np.diag(1./dd)  
        D_opt = D_opt / la.norm(D_opt)
        Domp = D_opt@W
        X_opt_train = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp.real, col=x), axis=0, arr=Y_train)
        X_opt_test = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp.real, col=x), axis=0, arr=Y_test)
        X_opt_train = W @ X_opt_train
        X_opt_test = W @ X_opt_test
        # Error Updating
        min_error_train_norm = (1/m_train)* np.sum(la.norm(Y_train - (D_opt @ X_opt_train), axis=0)**2 /
                                la.norm(Y_train, axis=0)**2)
        min_error_test_norm = (1/m_test)* np.sum(la.norm(Y_test - (D_opt @ X_opt_test), axis=0)**2 /
                                la.norm(Y_test, axis=0)**2)
        h_opt = 0
        
    return min_error_train_norm, min_error_test_norm, h_opt, X_opt_test, X_opt_train, D_opt

In [24]:
s = 8

In [25]:
c = c_true[s]  
epsilon = epsilon_true[s] 
k0 = K0_coll[1]

D0, X0, _ = initialize_dic(Lu, Ld, P, J, Y_train[:, :, s], k0, dictionary_type, c, epsilon, "only_X")

min_error_train_norm, min_error_test_norm, h_opt, X_opt_test, X_opt_train, D_opt = topological_dictionary_learn(Y_train[:,:,s], Y_test[:,:,s],
                                                                                                                        J, M, P, D0, X0, Lu, Ld, "separated",
                                                                                                                        c, epsilon, k0, lambda_, max_iter,
                                                                                                                        patience, tol)

In [27]:
h_opt

[array([[-0.00663524,  0.00051291],
        [ 0.00106015,  0.00128251],
        [ 0.00435605,  0.00100671]]),
 array([[-0.00205789,  0.00094752],
        [ 0.00015903,  0.00053683],
        [-0.0001982 ,  0.00022728]]),
 array([[0.14418037],
        [0.00123116],
        [0.00391908]])]

In [25]:
sigmas = pd.DataFrame({"b2": [B2]})
sigmas

,b2
0,"[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [28]:
def sparse_transform(D, K0, Y_test, Y_train=None):

    # OMP Step
    dd = la.norm(D, axis=0)
    W = np.diag(1. / dd)
    Domp = D @ W
    X_test = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_test)
    # Normalization
    X_test = W @ X_test

    # Same for the training set
    if Y_train!=None:
        X_train = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_train)
        X_train = W @ X_train

        return X_test, X_train
    
    return X_test

In [29]:
def nmse(D, X, Y, m):
    return (1/m)* np.sum(la.norm(Y - (D @ X), axis=0)**2 /la.norm(Y, axis=0)**2)

In [30]:
# global B2, h_opt, J

Ldk,_,_= compute_Lk_and_lambdak(Ld, J)


def indicator_matrix(row):
    row.sigma[row.idx] = 0
    return np.diag(row.sigma)

def compute_Luk(row, b2, J):
    Lu = b2 @ row.sigma @ b2.T
    Luk = np.array([la.matrix_power(Lu, i) for i in range(1, J + 1)])
    return Luk

T = B2.shape[1]
sigmas = pd.DataFrame({"idx": np.arange(T)})

sigmas["sigma"] = sigmas.idx.apply(lambda x: np.ones(T))
sigmas["sigma"] = sigmas.apply(lambda x: indicator_matrix(x), axis=1)
sigmas["Luk"] = sigmas.apply(lambda x: compute_Luk(x, B2, J), axis=1)
sigmas["D"] = sigmas.apply(lambda x: generate_dictionary(h_opt, 1, x.Luk, Ldk), axis=1)
sigmas["X"] = sigmas.D.apply(lambda x: sparse_transform(x, k0, Y_test[:,:,s]))
sigmas["NMSE"] = sigmas.apply(lambda x: nmse(x.D, x.X, Y_test[:,:,s], m_test), axis=1)

In [31]:
min_error_test_norm

0.041085929310523

In [34]:
sigmas.NMSE

0     0.213406
1     0.213309
2     0.213480
3     0.213293
4     0.213197
        ...   
57    0.213279
58    0.213148
59    0.213424
60    0.213142
61    0.212977
Name: NMSE, Length: 62, dtype: float64

In [66]:
sigmas.X[T-3].shape

(100, 80)